In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 우리가 사용할 컴퓨터를 check하는 부분, cpu/gpu 지원받을지...
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

# 1) 데이터셋 로딩 --- 2번에 걸쳐서 로딩
# train_dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/', # 데이터셋이 저장되는 물리적 위치
                                             train = True, # train dataset만
                                             transform = transforms.ToTensor(), # 중요!!
                                             download = True # 다운로드가 root에 진행됨
                                            )
test_dataset = torchvision.datasets.MNIST(root='../../data/', # 데이터셋이 저장되는 물리적 위치
                                             train = False, # test dataset만
                                             transform = transforms.ToTensor(), 
                                            )

# 2) DataLoader...BatchSize를 이용함
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Model 생성하기

In [ ]:
class NeuralNet(nn.Module):

    # 모델 설계
    def __init__(self,input_size, hidden_size, num_classes):
       
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    # 모델의 Forward Path를 정의
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)

        return out # 여기까지가 클래스 정의부분

# 위에서 정의한 클래스를 인스턴스화 시킴
model = NeuralNet(input_size, hidden_size, num_classes).to(device) # 이 model을 gpu 서버에서 돌리겠다

# loss and optimizer를 선정의
loss_function = nn.CrossEntropyLoss() # Loss 기능안에 Softmax 함수기능 포함되어져 있다.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_step = len(train_loader) # 600번 로딩..600번 학습이 진행된다.

for epoch in range(num_epochs): # 5번...
    for i, (images, labels) in enumerate(train_loader):
        # 네트워크에 넣어줄때는 1차원으로 펼쳐서 넣는다.
        images = images.reshape(-1, 28*28).to(device) # gpu로 돌린다.
        labels = labels.to(device)

        # Forward Pass
        pred = model(images)
        loss = loss_function(pred,labels)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        if(i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 0.3694
Epoch [1/5], Step [200/600], Loss: 0.2846
Epoch [1/5], Step [300/600], Loss: 0.1711
Epoch [1/5], Step [400/600], Loss: 0.1277
Epoch [1/5], Step [500/600], Loss: 0.1217
Epoch [1/5], Step [600/600], Loss: 0.0595
Epoch [2/5], Step [100/600], Loss: 0.1639
Epoch [2/5], Step [200/600], Loss: 0.1255
Epoch [2/5], Step [300/600], Loss: 0.1884
Epoch [2/5], Step [400/600], Loss: 0.1428
Epoch [2/5], Step [500/600], Loss: 0.0742
Epoch [2/5], Step [600/600], Loss: 0.0621
Epoch [3/5], Step [100/600], Loss: 0.1481
Epoch [3/5], Step [200/600], Loss: 0.1188
Epoch [3/5], Step [300/600], Loss: 0.0455
Epoch [3/5], Step [400/600], Loss: 0.0501
Epoch [3/5], Step [500/600], Loss: 0.0832
Epoch [3/5], Step [600/600], Loss: 0.0562
Epoch [4/5], Step [100/600], Loss: 0.0546
Epoch [4/5], Step [200/600], Loss: 0.0507
Epoch [4/5], Step [300/600], Loss: 0.0933
Epoch [4/5], Step [400/600], Loss: 0.0704
Epoch [4/5], Step [500/600], Loss: 0.0765
Epoch [4/5], Step [600/600], Loss:

In [ ]:
## Test Model에서는 필요한 구문이다.
'''
1) with torch.no_grad():
    some code
    --> backward를 하지 않겠다.
2) with torch set_grad_enabled(False)

구문이 필요한 이유가
안해주면 기본적으로 BackPropagation을 진행할 것으로 알고 이에 해당하는 메모리를
따로 빼둔다.
결과적으로 이 구문을 작성하지 않으면 메모리를 더 많이 먹는다.

'''
with torch.no_grad(): #실제로 
    correct = 0
    total = 0

    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)

        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Accuracy of the Network on the Test Images :{} %'.format(100 * correct/total))
torch.save(model.state_dict(), 'model.ckpt')

Accuracy of the Network on the Test Images :97.93 %
